## 0-Libraries

In [1]:
from Functions.sql_functions import *

# 1- Creating the DataBase

## 1.1 Creating The Tables

In [2]:
sql1 = "DROP TABLE IF EXISTS Item"
sql_write(sql1)

sql2 = '''
  CREATE TABLE Item (
    Item_ID INTEGER NOT NULL PRIMARY KEY,
    Item_Name TEXT,
    Lot_Size INTEGER,
    Lead_Time INTEGER,
    Current_Inventory INTEGER,
    BOM_Level INTEGER
  );
'''
sql_write(sql2)

In [3]:
sql1 = "DROP TABLE IF EXISTS BOM"
sql_write(sql1)

sql2 = '''
  CREATE TABLE BOM (
    Item_ID INTEGER NOT NULL,
    Component_ID INTEGER NOT NULL,
    BOM_Multiplier INTEGER,
    PRIMARY KEY(Item_ID, Component_ID),
    FOREIGN KEY(Item_ID) REFERENCES Item(Item_ID),
    FOREIGN KEY(Component_ID) REFERENCES Item(Item_ID)
  );
'''
sql_write(sql2)

In [4]:
sql1 = "DROP TABLE IF EXISTS Period"
sql_write(sql1)

sql2 = '''
    CREATE TABLE Period (
        Period_ID INTEGER NOT NULL PRIMARY KEY,
        Date TEXT
    );
'''
sql_write(sql2)

In [5]:
sql1 = "DROP TABLE IF EXISTS Item_Period"
sql_write(sql1)

sql2 = '''
  CREATE TABLE Item_Period (
    Item_ID INTEGER NOT NULL,
    Period_ID INTEGER NOT NULL,
    Gross_Requirement INTEGER,
    Scheduled_Receipt INTEGER,
    Projected_Inventory INTEGER,
    Net_Requirement INTEGER,
    Planned_Order_Receipt INTEGER,
    Planned_Order_Release INTEGER,
    PRIMARY KEY (Item_ID, Period_ID),
    FOREIGN KEY(Item_ID) REFERENCES Item(Item_ID),
    FOREIGN KEY(Period_ID) REFERENCES Period(Period_ID)
  );
'''
sql_write(sql2)

## 1.2- Filling the Tables

In [6]:
from Functions.table_data import excel_to_table

### 1.2.1- Inserting Data using excel files

In [7]:
empty_table('Item')
excel_to_table(ref='Data/Item_data.xlsx', table='Item')

In [8]:
empty_table('BOM')
excel_to_table(ref='Data/BOM_data.xlsx', table='BOM')

In [9]:
empty_table('Period')
excel_to_table(ref='Data/Period_data.xlsx', table='Period')

In [10]:
empty_table('Item_Period')
excel_to_table(ref='Data/Item_Period_data.xlsx', table='Item_Period')

# 2- Calculating BOM levels and materials

In [11]:
from Functions.mrp_sql_functions import sql_update_bom_levels

In [12]:
sql_update_bom_levels()

# 3- MRP calculations

In [13]:
from Functions.mrp_sql_functions import *

In [14]:
calc_all_mrp_table()

# 4- Compare results

### 4.1- MRP sql calculations

In [15]:
import pandas as pd

In [16]:
sql = '''
    SELECT * FROM Item_Period
'''
pd.DataFrame(sql_read(sql), columns=['Item_ID', 'Period_ID', 'Gross_Requirement', 'Scheduled_Receipt', 'Projected_Inventory', 'Net_Requirement', 'Planned_Order_Receipt', 'Planned_Order_Release'])

,Item_ID,Period_ID,Gross_Requirement,Scheduled_Receipt,Projected_Inventory,Net_Requirement,Planned_Order_Receipt,Planned_Order_Release
0,1,1,6,0,2,0,0,0.0
1,1,2,4,10,8,0,0,0.0
2,1,3,15,0,3,7,10,10.0
3,1,4,9,0,4,6,10,10.0
4,1,5,7,0,7,3,10,10.0
5,2,1,6,25,69,0,0,25.0
6,2,2,4,0,65,0,0,25.0
7,2,3,42,0,23,0,0,NaN
8,2,4,30,0,18,7,25,NaN
9,2,5,23,0,20,5,25,NaN


### 4.2- Hand calculations

In [17]:
pd.read_excel('Data/hand_solution/hand_solution.xlsx')

,Item_ID,Period_ID,Gross_Requirement,Scheduled_Receipt,Projected_inventory,Net_Requirement,Planned_Order_Receipt,Planned_Order_Release
0,1,1,6,0,2,0,0,0
1,1,2,4,10,8,0,0,0
2,1,3,15,0,3,7,10,10
3,1,4,9,0,4,6,10,10
4,1,5,7,0,7,3,10,10
5,2,1,6,25,69,0,0,25
6,2,2,4,0,65,0,0,25
7,2,3,22,0,23,0,0,0
8,2,4,10,0,18,7,25,0
9,2,5,3,0,20,5,25,0
